In [ ]:
import pandas as pd
import os
from iteround import saferound
import geopandas as gpd

from util.census_api import CensusApi

api_key = os.getenv('CENSUS_KEY','no variable found')
print(f'API Key: {api_key}')

county_ids = [53033,53035,53053,53061]
state_id = 53

c = CensusApi(api_key)

In [ ]:
cols_dict = {
        'total_population': {
            'race': ['hispanic','white_nh','black_nh','aian_nh','asian_pac_nh','other_nh'],
            #'age': ['age_under5','age_5to17','age_18to24','age_25to44','age_45to64','age_65plus']
        },

        'total_households': {

        }
}

# create a flat list of all columns to retrieve
cols_to_get = []
for key in cols_dict:
    for subkey in cols_dict[key]:
        cols_to_get.extend(cols_dict[key][subkey])

cols_to_get

In [ ]:
def xwalk_merge_sum(xwalk,xwalk_left_index,xwalk_right_index,df,df_index,wt_col,col_list,total_col):
    '''
    xwalk: crosswalk dataframe
    xwalk_left_index: column in xwalk to merge on (left)
    xwalk_right_index: column in xwalk to group by (right)
    df: dataframe to merge (right)
    df_index: column in df to merge on (right)
    wt_col: column in xwalk with weights
    col_list: list of columns to sum
    total_col: column with total
    '''
    xwalk = xwalk.merge(df, left_on=xwalk_left_index, right_on=df_index, how='left')
    
    for col in col_list + [total_col]:
        xwalk[col] = xwalk[col] * xwalk[wt_col]
    xwalk = xwalk[col_list + [xwalk_right_index, total_col]].groupby(xwalk_right_index).sum()

    return xwalk.reset_index()

In [ ]:
def normalize_round(df,index_col, total_col):
    '''
    df: dataframe to normalize
    index_col: column with geography identifier
    total_col: column with total, ie. total_population or total_households
    '''
    # create a dataframe with total for each tract
    tot_df = df[[index_col,total_col]].copy().rename(columns={total_col:'total'}).set_index(index_col)
    tot_df['total'] = saferound(tot_df['total'],0)

    # reshape to tidy format and calculate percentages
    tidy_df = df.drop(columns=[total_col]).melt(id_vars=[index_col], value_vars=cols_to_get, var_name='variable', value_name='total')
    tidy_df = tidy_df.set_index(index_col)
    tidy_df['pct'] = tidy_df['total'] / tidy_df.groupby(index_col)['total'].transform('sum')
    tidy_df = tidy_df.drop(columns=['total'])

    # merge actual total back in and calculate new counts
    tidy_df = tidy_df.merge(tot_df,how='left', left_index=True, right_index=True)
    
    # normalize
    tidy_df['new_total'] =  tidy_df['pct'] * tidy_df['total']
    
    # round within each geography
    for geog in tidy_df.index.unique():
        mask = tidy_df.index == geog
        if tidy_df.loc[mask, 'pct'].sum() > 0:
            tidy_df.loc[mask, 'new_total'] = saferound(tidy_df.loc[mask, 'new_total'],0)
    
    # cleanup dataframe
    tidy_df = tidy_df.drop(columns=['pct','total']).rename(columns={'new_total':total_col})
    
    # reshape back to wide format
    result = tidy_df.reset_index().pivot(index=index_col, columns='variable', values=total_col)
    result.columns.name = None  # remove columns' name for cleaner output
    result[total_col] = result.sum(axis=1)
    return result.fillna(0).astype(int).reset_index()

##### 1990 Decennial

In [ ]:
# load 1990 data from ipums
df90 = pd.read_csv('data/nhgis0017_ds120_1990_tract.csv')


df90['hispanic'] = df90[['ET2006','ET2007','ET2008','ET2009','ET2010']].sum(axis=1)

df90 = df90.rename(columns = {
    'ET1001':'total_population',
    'ET2001':'white_nh',
    'ET2002':'black_nh',
    'ET2003':'aian_nh',
    'ET2004':'asian_pac_nh',
    'ET2005':'other_nh'
})

df90 = df90[['GISJOIN','total_population'] + cols_to_get].copy()

# check totals
print(f"total population: {int(df90['total_population'].sum()):,}")
print(f"race col total: {int(df90[cols_to_get].sum().sum()):,}")
print("If these two numbers match, then the race variables were summed correctly.")

In [ ]:
# load crosswalk from 1990 tracts to 2010 tracts
xwalk90 = (
    pd.read_csv('xwalks/nhgis_tr1990_tr2010_53.csv')
)

In [ ]:
# perform crosswalk merge and sum for 1990 data to 2010 tracts
out90 = xwalk_merge_sum(xwalk90, 'tr1990gj','tr2010ge',df90,'GISJOIN','wt_pop',cols_to_get,'total_population')

In [ ]:
# load crosswalk from 2010 tracts to 2020 tracts
xwalk10 = (
    pd.read_csv('xwalks/nhgis_tr2010_tr2020_53.csv')
)

In [ ]:
# perform crosswalk merge and sum the 1990 data from 2010 to 2020 tracts
out90_to_20 = xwalk_merge_sum(xwalk10, 'tr2010ge','tr2020ge',out90,'tr2010ge','wt_pop',cols_to_get,'total_population')

# normalize and round
out90_to_20 = normalize_round(out90_to_20,'tr2020ge','total_population')

# filter to 4 counties
out90_to_20['county_id'] = out90_to_20['tr2020ge'].astype(str).str[0:5].astype(int)
out90_to_20 = out90_to_20.loc[out90_to_20.county_id.isin(county_ids)].copy()

# add year column
out90_to_20['year'] = 1990

print(f"total population: {int(out90_to_20['total_population'].sum()):,}")
print(f"race col total: {int(out90_to_20[cols_to_get].sum().sum()):,}")

##### 2000 Decennial

In [ ]:
# 2000 decennial data
variables_dict = {
    'total_population': ['P001001'],
    'hispanic':['P004002'],
    'white_nh':['P004005'],
    'black_nh':['P004006'],
    'aian_nh':['P004007'],
    'asian_pac_nh':['P004008','P004009'],
    'other_nh':['P004010','P004011']
}

d00 = c.get_dec_data(variables_dict, 2000,'tract','sf1', county_ids, state_id)

# check totals
print(f"total population: {int(d00['total_population'].sum()):,}")
print(f"race col total: {int(d00[cols_to_get].sum().sum()):,}")
print("If these two numbers match, then the race variables were summed correctly.")

In [ ]:
xwalk00 = (
    pd.read_csv('xwalks/nhgis_tr2000_tr2010_53.csv')
)

In [ ]:
# perform crosswalk merge and sum for 2000 data to 2010 tracts
out00_to_10 = xwalk_merge_sum(xwalk00, 'tr2000ge','tr2010ge',d00,'geoid','wt_pop',cols_to_get,'total_population')

print(f"total population: {int(out00_to_10['total_population'].sum()):,}")
print(f"race col total: {int(out00_to_10[cols_to_get].sum().sum()):,}")

In [ ]:
# perform crosswalk merge and sum for 2000 data to 2010 tracts
out00_to_20 = xwalk_merge_sum(xwalk10, 'tr2010ge','tr2020ge',out00_to_10,'tr2010ge','wt_pop',cols_to_get,'total_population')

# normalize and round
out00_to_20 = normalize_round(out00_to_20,'tr2020ge','total_population')

# filter to four counties
out00_to_20['county_id'] = out00_to_20['tr2020ge'].astype(str).str[0:5].astype(int)
out00_to_20 = out00_to_20.loc[out00_to_20.county_id.isin(county_ids)].copy()

# add year column
out00_to_20['year'] = 2000

print(f"total population: {int(out00_to_20['total_population'].sum()):,}")
print(f"race col total: {int(out00_to_20[cols_to_get].sum().sum()):,}")

##### 2010 Decennial

In [ ]:
# 2010 decennial data
variables_dict = {
    'total_population': ['P001001'],
    'hispanic':['P009002'],
    'white_nh':['P009005'],
    'black_nh':['P009006'],
    'aian_nh':['P009007'],
    'asian_pac_nh':['P009008','P009009'],
    'other_nh':['P009010','P009011']
}

df10 = c.get_dec_data(variables_dict, 2010,'tract','sf1', county_ids, state_id)

# check totals
print(f"total population: {int(df10['total_population'].sum()):,}")
print(f"race col total: {int(df10[cols_to_get].sum().sum()):,}")
print("If these two numbers match, then the race variables were summed correctly.")

In [ ]:
# perform crosswalk merge and sum for 2010 data to 2020 tracts
out10_to_20 = xwalk_merge_sum(xwalk10, 'tr2010ge','tr2020ge',df10,'geoid','wt_pop',cols_to_get,'total_population')

# normalize and round
out10_to_20 = normalize_round(out10_to_20,'tr2020ge','total_population')

# filter to four counties
out10_to_20['county_id'] = out10_to_20['tr2020ge'].astype(str).str[0:5].astype(int)
out10_to_20 = out10_to_20.loc[out10_to_20.county_id.isin(county_ids)].copy()

# add year column
out10_to_20['year'] = 2010

print(f"total population: {int(out10_to_20['total_population'].sum()):,}")
print(f"race col total: {int(out10_to_20[cols_to_get].sum().sum()):,}")

##### 2020 Decennial

In [ ]:
# 2020 decennial data
variables_dict = {
    'total_population': ['P1_001N'],
    'hispanic':['P9_002N'],
    'white_nh':['P9_005N'],
    'black_nh':['P9_006N'],
    'aian_nh':['P9_007N'],
    'asian_pac_nh':['P9_008N','P9_009N'],
    'other_nh':['P9_010N','P9_011N']
}

df20 = (
    c.get_dec_data(variables_dict, 2020,'tract','dhc', county_ids, state_id)
    .drop(columns='name')
    .rename(columns={'geoid':'tr2020ge'})
    .assign(county_id=lambda x: x['tr2020ge'].astype(str).str[0:5].astype(int),
            year=2020)
    .set_index('tr2020ge').astype(int).reset_index()
)


# check totals
print(f"total population: {int(df20['total_population'].sum()):,}")
print(f"race col total: {int(df20[cols_to_get].sum().sum()):,}")
print("If these two numbers match, then the race variables were summed correctly.")

In [ ]:
df = pd.concat([
    out90_to_20,
    out00_to_20,
    out10_to_20,
    df20
], ignore_index=True)

In [ ]:
df['poc'] = df['total_population'] - df['white_nh']

In [ ]:
today = pd.Timestamp('today').strftime('%Y_%m%d')

df[['tr2020ge','county_id','year'] + cols_to_get + ['poc']].to_csv(f'output/{today}_reap_decennial.csv', index=False)

In [ ]:
piv = df[['tr2020ge','poc','white_nh','total_population','year']].pivot(index='tr2020ge', columns='year', values=['poc','white_nh','total_population'])

# flatten multiindex columns
piv.columns = [f'{col[0]}_{col[1]}' for col in piv.columns]
piv = piv.reset_index()

In [ ]:
piv['poc_diff_1990_2020'] = piv['poc_2020'] - piv['poc_1990']
piv['white_nh_diff_1990_2020'] = piv['white_nh_2020'] - piv['white_nh_1990']
piv['total_population_diff_1990_2020'] = piv['total_population_2020'] - piv['total_population_1990']

In [ ]:
tracts = (
    gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2025/TRACT/tl_2025_53_tract.zip')
    [['GEOID','geometry']]
)

tracts['tr2020ge'] = tracts['GEOID'].astype('int64')
tracts = tracts.drop(columns=['GEOID'])

In [ ]:
tracts.merge(piv, on='tr2020ge', how='inner').explore(column='poc_diff_1990_2020',scheme='percentiles').save('output/2025_1231_reap_dec_poc_diff.html')